## SQL USED TO CREATE META COHORT OF 5M CUSTOMERS

In [ ]:
sql = """ 

set periodEnddate = '2024-05-30';
set stly_start = '2023-05-06';
set stly_end = '2023-11-05';


create or replace table mrch_sandbox.loyalty_eda as
WITH ACTIVE_AS AS 
(
SELECT DISTINCT CUSTOMER_ID, STATUS  
FROM "EDLDB"."CDM"."SUBSCRIPTIONS_SNAPSHOT" 
WHERE SNAPSHOT_DATE = date($periodEnddate) 
AND STATUS = 'Active'
),

customers_selected as (
select
a.customer_id,
case when b.STATUS = 'Active' then True else False end as Active_AS_STATUS,
MIN(CASE WHEN ORDER_FIRST_ORDER_PLACED_FLAG = TRUE  THEN DATE(ORDER_PLACED_DTTM) END) AS ACQUISITION_DATE,
MAX(CASE WHEN ORDER_SUBMITTED_BY = 'SYSTEM' THEN DATE(ORDER_PLACED_DTTM) END) AS LAST_AS_ORDER_DATE,
COUNT(DISTINCT CASE WHEN ORDER_SUBMITTED_BY = 'SYSTEM' AND DATE(ORDER_PLACED_DTTM_EST) BETWEEN date($periodEnddate)-89 AND $periodEnddate THEN ORDER_ID END) AS L90_AS_ORDERS,

count(distinct case when DATE(ORDER_PLACED_DTTM) between date($periodEnddate)-89 and $periodEnddate  then order_id end) as orders_l90,

count(distinct case when DATE(ORDER_PLACED_DTTM) between date($periodEnddate)-364 and $periodEnddate  then order_id end) as orders_ly,
sum(case when DATE(ORDER_PLACED_DTTM) between date($periodEnddate)-364 and $periodEnddate  then net_sales end) as net_sales_ly,

count(distinct case when DATE(ORDER_PLACED_DTTM) between $stly_start and $stly_end then order_id end ) as orders_stly,
sum(case when DATE(ORDER_PLACED_DTTM) between $stly_start and $stly_end then net_sales end) as net_sales_stly,

from ecom.order_line a
left join ACTIVE_AS b on a.customer_id = b.customer_id
where 1=1
    AND ORDER_STATUS NOT IN ('J','I','X','P')
    AND ORDER_LINE_TOTAL_PRICE !=0
    AND DATE(ORDER_PLACED_DTTM) <=  $periodEnddate
group by 1,2
),
update_data as (
select 
customer_id,
ACQUISITION_DATE,
date($periodEnddate) - ACQUISITION_DATE + 1 AS TENURE,
case when Active_AS_STATUS = False and L90_AS_ORDERS = 0 then 0 else 1 end as Active_AS_Flag,
ifnull(net_sales_ly , 0) as net_sales_ly,
ifnull(net_sales_stly , 0) as net_sales_stly,
ifnull(orders_l90 , 0) as orders_l90,
ifnull(orders_ly , 0) as orders_ly,
ifnull(orders_stly , 0) as orders_stly,
from customers_selected

)
select 
*,
case when round(net_sales_ly, 0) <= 0 then 'ns_0'
when round(net_sales_ly, 0) between 1 and 50 then 'ns_1_50'
when round(net_sales_ly, 0) between 51 and 150 then 'ns_51_150'
when round(net_sales_ly, 0) between 151 and 250 then 'ns_150_250'
when round(net_sales_ly, 0) between 251 and 300 then 'ns_250_300'
when round(net_sales_ly, 0) between 301 and 500 then 'ns_300_500'
when round(net_sales_ly, 0) between 501 and 700 then 'ns_500_700'
when round(net_sales_ly, 0) between 701 and 1100 then 'ns_700_1100'
when round(net_sales_ly, 0) between 1101 and 1200 then 'ns_1100_1200'
when round(net_sales_ly, 0) between 1201 and 1500 then 'ns_1200_1500'
when round(net_sales_ly, 0) > 1500 then 'ns_gt_1500'
end as net_sales_ly_buckets

from update_data;


----------

create or replace table mrch_sandbox.MRCH_DS_loyalty_ec_eda as
with base_segment_table as (
select
distinct 
session_date,
customer_id,
sum(HOME_PAGE_IMPRESSION_HITS) as total_hpit
from EDLDB.ECOM_SANDBOX.HEARTBEAT_SESSION_AGGREGATE
where session_date >= '2024-01-01'
and customer_id in (select distinct customer_id from mrch_sandbox.loyalty_eda )
group by 1,2
),
customer_data as (
select 
a.*,
b.* exclude(customer_id)
from base_segment_table a
left join  mrch_sandbox.loyalty_eda b on a.customer_id = b.customer_id
)
select
session_date, 
count(distinct case when ACQUISITION_DATE < session_date then customer_id end) as ec,
count(distinct case when  TENURE>365 then customer_id end) as ec_tenure,
count(distinct case when  TENURE>365  and  Active_AS_Flag = 0 then customer_id end) as ec_tenure_non_as,
count(distinct case when  TENURE>365  and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_0' then customer_id end)  as ec_ns_0,
count(distinct case when  TENURE>365  and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_1_50' then customer_id end)  as ec_ns_1_50,
count(distinct case when  TENURE>365  and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_51_150' then customer_id end)  as ec_ns_51_150,
count(distinct case when  TENURE>365  and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_150_250' then customer_id end)  as ec_ns_150_250,
count(distinct case when  TENURE>365  and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_250_300' then customer_id end)  as ec_ns_250_300,
count(distinct case when  TENURE>365  and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_300_500' then customer_id end)  as ec_ns_300_500,
count(distinct case when  TENURE>365  and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_500_700' then customer_id end)  as ec_ns_500_700,
count(distinct case when  TENURE>365  and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_700_1100' then customer_id end) as ec_ns_700_1100,
count(distinct case when  TENURE>365  and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_1100_1200' then customer_id end) as ec_ns_1100_1200,
count(distinct case when  TENURE>365  and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_1200_1500' then customer_id end) as ec_ns_1200_1500,
count(distinct case when  TENURE>365  and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_gt_1500' then customer_id end) as ec_ns_gt_1500,

count(distinct case when total_hpit >0  and  ACQUISITION_DATE < session_date then customer_id end) as hp_ec,
count(distinct case when  TENURE>365 and  total_hpit >0 then customer_id end) as hp_ec_tenure,
count(distinct case when  TENURE>365  and  total_hpit >0 and  Active_AS_Flag = 0 then customer_id end) as hp_ec_tenure_non_as,
count(distinct case when  TENURE>365  and  total_hpit >0 and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_0' then customer_id end)  as hp_ec_ns_0,
count(distinct case when  TENURE>365  and  total_hpit >0 and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_1_50' then customer_id end)  as hp_ec_ns_1_50,
count(distinct case when  TENURE>365  and  total_hpit >0 and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_51_150' then customer_id end)  as hp_ec_ns_51_150,
count(distinct case when  TENURE>365  and  total_hpit >0 and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_150_250' then customer_id end) as hp_ec_ns_150_250,
count(distinct case when  TENURE>365  and  total_hpit >0 and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_250_300' then customer_id end) as hp_ec_ns_250_300,
count(distinct case when  TENURE>365  and  total_hpit >0 and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_300_500' then customer_id end) as hp_ec_ns_300_500,
count(distinct case when  TENURE>365  and  total_hpit >0 and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_500_700' then customer_id end) as hp_ec_ns_500_700,
count(distinct case when  TENURE>365  and  total_hpit >0 and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_700_1100' then customer_id end) as hp_ec_ns_700_1100,
count(distinct case when  TENURE>365  and  total_hpit >0 and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_1100_1200' then customer_id end) as hp_ec_ns_1100_1200,
count(distinct case when  TENURE>365  and  total_hpit >0 and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_1200_1500' then customer_id end) as hp_ec_ns_1200_1500,
count(distinct case when  TENURE>365  and  total_hpit >0 and  Active_AS_Flag = 0 and net_sales_ly_buckets = 'ns_gt_1500' then customer_id end) as hp_ec_ns_gt_1500

from customer_data
group by 1;



-------


;


create or replace table mrch_sandbox.mrch_ds_loyalty_customer_data as 
with cte1_1 as (
select 
net_sales_ly_buckets,
min(net_sales_ly) as start_bucket,
max(net_sales_ly) as end_bucket,
from mrch_sandbox.loyalty_eda
where net_sales_ly_buckets not in ('ns_0' , 'ns_gt_1500')
group by 1

),
cte1_2 as (

select 
*,
rank() over(order by start_bucket asc) as rank_buckets
from cte1_1
),
cte2 as (
select 
b.*,
concat(rank_buckets , '_' , a.NET_SALES_LY_BUCKETS) as buckets,
case when net_sales_stly between 0.1 and end_bucket then 1 else 0 end as CONSISTENT_FLAG,
case when ORDERS_L90 >= 1 and ORDERS_STLY >= 2 then 1 else 0 end as REGULAR_FLAG
from cte1_2 a
left join mrch_sandbox.loyalty_eda b on  a.net_sales_ly_buckets = b.net_sales_ly_buckets
where TENURE>365  and  Active_AS_Flag = 0 
),
cte4 as (
select 
distinct buckets ,  
REGULAR_FLAG , 
CONSISTENT_FLAG ,
count(distinct customer_id) over(partition by buckets , REGULAR_FLAG, CONSISTENT_FLAG ) customer_count,
count(distinct customer_id) over(partition by buckets  ) total_customers
from cte2
)
select 
a.*,
case when customer_count>= 20000 or round(customer_count / total_customers, 2) >= 0.05 then 1 else 0 end as CONSIDER_FLAG
from cte2 a 
right join cte4 b on a.buckets = b.buckets and a.REGULAR_FLAG = b.REGULAR_FLAG and a.CONSISTENT_FLAG = b.CONSISTENT_FLAG
;

-----------


select *
from mrch_sandbox.mrch_ds_loyalty_customer_data
group by 1;

-----


set periodEnddate = '2024-05-30';
set stly_start = '2023-05-06';
set stly_end = '2023-11-05';


create or replace table mrch_sandbox.mrch_ds_customer_segment_data as
WITH CTE AS (SELECT 
    CUSTOMER_ID, 
    MIN(CASE WHEN ORDER_AUTO_REORDER_FLAG = TRUE THEN DATE(ORDER_PLACED_DTTM) END) AS AS_ACQUISITION_DATE,
FROM 
    "EDLDB"."ECOM"."ORDERS" 
WHERE 
    1=1
    AND ORDER_STATUS NOT IN ('J','I','X','P')
    AND DATE(ORDER_PLACED_DTTM) <= $periodEnddate
    AND CUSTOMER_ID IN (SELECT DISTINCT CUSTOMER_ID FROM mrch_sandbox.mrch_ds_loyalty_customer_data)
GROUP BY 1
),
autoship_data as (
SELECT
    DISTINCT CUSTOMER_ID,
    (CASE WHEN AS_ACQUISITION_DATE IS NOT NULL then 1 else 0 end) as AS_FLAG
FROM CTE
),

// PET PROFILE
pet_data_CTE AS (
    SELECT 
        a.CUSTOMER_ID,
        COUNT (DISTINCT b.PET_PROFILE_ID) AS PET_COUNT
    FROM mrch_sandbox.mrch_ds_loyalty_customer_data a 
    LEFT JOIN "EDLDB"."CDM"."PET_PROFILE" b 
    ON a.CUSTOMER_ID = b.CUSTOMER_ID 
    GROUP BY 1
),
pet_data as (
SELECT
    CUSTOMER_ID,
    CASE WHEN PET_COUNT = 1 THEN 'SINGLE_PET' 
         WHEN PET_COUNT >1 THEN 'MULTI_PET' 
    ELSE 'NO_PET'
    END as PET_FLAG
FROM
    pet_data_CTE
),

----mc2
mc2_cte as (
SELECT 
    distinct CUSTOMER_ID,
    count (distinct case when DATE(ORDER_PLACED_DTTM_EST) BETWEEN $stly_start AND $stly_end then MERCH_CLASSIFICATION2 end) as MC2_COUNT_LYTP
       
FROM ECOM.ORDER_LINE a
LEFT JOIN EDLDB.PDM.PRODUCT b on a.PART_NUMBER = b.PART_NUMBER  
where  1=1
AND CUSTOMER_ID in (select distinct customer_id from mrch_sandbox.mrch_ds_loyalty_customer_data)
AND order_order_line_status NOT IN ('J','X') 
AND ORDER_LINE_TOTAL_PRICE !=0 
group by 1

),
mc2 as (
select     
    DISTINCT customer_id,
    case when MC2_COUNT_LYTP = 1 then 1 else 0 end as MC2_FLAG
from  mc2_cte 
),
promo_cte as (
SELECT 
        distinct CUSTOMER_ID,
        count (distinct case when DATE(ORDER_PLACED_DTTM) BETWEEN $stly_start AND $stly_end  then promotion_id  end) as PROMO_COUNT_LYTP,

FROM cdm.order_line_promotion_usage 
      
where  1=1
AND CUSTOMER_ID in (select distinct customer_id from  mrch_sandbox.mrch_ds_loyalty_customer_data)
AND order_status NOT IN ('J','X' , 'I' , 'P') 
and promotion_id in  (9531, 9938, 10425, 10629, 10675, 10891, 11217, 11289, 11570 , 11745 )
group by 1

),
promo_data as (

select     
    DISTINCT customer_id,
    CASE WHEN PROMO_COUNT_LYTP = 1 THEN 'SINGLE_PROMO' 
         WHEN PROMO_COUNT_LYTP >1 THEN 'MULTI_PROMO' 
        ELSE 'NO_PROMO'
    END as SW_PROMO_FLAG
    
from promo_cte
)

select 
DISTINCT 
a.buckets ,  
a.REGULAR_FLAG , 
a.CONSISTENT_FLAG ,
a.CUSTOMER_ID,
b.AS_FLAG,
d.PET_FLAG,
e.MC2_FLAG,
ifnull(f.SW_PROMO_FLAG , 'NO_PROMO') as SW_PROMO_FLAG


from mrch_sandbox.mrch_ds_loyalty_customer_data a
left join autoship_data b on a.customer_id = b.customer_id
left join pet_data d on a.customer_id = d.customer_id
left join mc2 e on a.customer_id = e.customer_id
left join promo_data f on a.customer_id = f.customer_id
where a.CONSIDER_FLAG = 1
;



select 
buckets,
REGULAR_FLAG,
CONSISTENT_FLAG,
percentile_cont(0.1) within group (  order by a.tenure),
percentile_cont(0.25) within group ( order by a.tenure),
percentile_cont(0.50) within group ( order by a.tenure),
percentile_cont(0.75) within group ( order by a.tenure),
percentile_cont(0.9) within group ( order by a.tenure)

from mrch_sandbox.mrch_ds_loyalty_customer_data a 
where REGULAR_FLAG = 1
and CONSISTENT_FLAG = 1
and buckets = '4_ns_250_300'
group by 1,2,3
order by 1;



---------




"""

## SQL TO IDENTIFY HOW MANY CUSTOMERS VISITED AND PLACED AN ORDER

In [ ]:
sql = """  

set start_date = '2024-06-20';
set end_date = '2024-06-24';


-- select * from  MRCH_SANDBOX.MRCH_DS_LOYALTY_CUST_SEGMENT_META_COHORT_UPDATED
-- where customer_id in (111355305)

with wave1_cust as (
select distinct * from 
MRCH_SANDBOX.MRCH_DS_LOYALTY_CUST_SEGMENT_META_COHORT_UPDATED
where updated_flag = 'Control'
),
order_check as (

select customer_id,
count(distinct order_id) order_count
from ecom.orders
where 1 =1 
AND order_submitted_by = 'CUSTOMER'
AND DATE(ORDER_PLACED_DTTM) between $start_date and $end_date
and customer_id in  (select distinct customer_id from wave1_cust)
group by 1
having order_count>=1
),

unique_users as (
select 
customer_id, 
count(distinct session_id) as session_cnt,
count(distinct case when HOME_PAGE_IMPRESSION_HITS >0 then session_id end ) impression_sessions
from EDLDB.ECOM_SANDBOX.HEARTBEAT_SESSION_AGGREGATE
where session_date between $start_date and $end_date
and customer_id in (select distinct customer_id from wave1_cust)
group by 1
)


select
a.* ,
ifnull(b.order_count , 0) as order_count
from unique_users a 
left join order_check b on a.customer_id = b.customer_id 

where order_count !=0 ;
-- where IMPRESSION_SESSIONS >0 ;


"""

## SQL TO CREATE CONTROL AND TEST GROUPS

In [ ]:
sql = """ 

##### TEST CUSTOMER ANALYSIS  ######

with cte1 as (
select *,
case when c.cohort_description is null then 'Not Available' else c.cohort_description end as cohort
from edldb.loyalty_sandbox.master_test_cohort c 
where friend_family_indicator = FALSE
),
customer_base as (
select *
from cte1  
where cohort != 'Invite Request' 
),
signup_customer as (
select
distinct c.customer_id,
date(created_timestamp) as s_date
from edldb.chewybi.membership_metrics m
left join customer_base c
on m.customer_id = c.customer_id
where 1=1
and m.membership_status = 'ACTIVE'
and plan_type = 'FREE_TRIAL'
),
flags_added as (
select
distinct b.* , 
c.regular_flag, c.consistent_flag, c.buckets , c.updated_flag 
from signup_customer b
left join MRCH_SANDBOX.MRCH_DS_LOYALTY_CUST_SEGMENT_META_COHORT_UPDATED c on  b.customer_id = c.customer_id 
where b.customer_id is not null
),
final_test_cust as (
select
distinct b.* ,
a.cohort_description  as cohort_type
from flags_added b
left join  LOYALTY_SANDBOX.CUSTOMER_SEGMENT a on b.customer_id = a.customer_id and WAVE_ID != 5
),
order_check as (
select 
distinct a.customer_id,
count(distinct order_id) order_count
from final_test_cust a
left join ecom.orders on a.customer_id = b.customer_id and DATE(ORDER_PLACED_DTTM) = a.s_date
where 1 =1 
-- AND order_submitted_by = 'CUSTOMER'
and order_status not in ('J' , 'I' , 'X' , 'P') 
group by 1
having order_count>=1
),
promo_order as (
select customer_id
from cdm.order_line_promotion_usage
where promotion_id = 12105
and customer_id in  (select distinct customer_id from final_test_cust )
and order_status not in ('J' , 'I' , 'X' , 'P') 
),
flags_test_data as (
select distinct a.*,
case when d.customer_id is null then False else True end as SIGNUP_ORDER_FLAG,
case when e.customer_id is null then False else True end as SW_ORDER_FLAG,
case when s_date between '2024-06-17' and '2024-06-27' then True else False end as SW_ACQUIRED

from final_test_cust a
left join order_check d on a.customer_id = d.customer_id
left join promo_order e on a.customer_id = e.customer_id
)
select * from flags_test_data


;

------------


##### CONTROL CUSTOMER ANALYSIS  ######


set start_date = '2024-05-26';
set end_date = '2024-06-26';

with control_customer as (
select distinct * from 
MRCH_SANDBOX.MRCH_DS_LOYALTY_CUST_SEGMENT_META_COHORT_UPDATED
where updated_flag = 'Control'
),
order_check as (
select 
distinct customer_id,
count(distinct order_id) order_count
from ecom.orders
where 1 =1 
-- AND order_submitted_by = 'CUSTOMER'
AND DATE(ORDER_PLACED_DTTM) between $start_date and $end_date
and customer_id in  (select distinct customer_id from control_customer)
and order_status not in ('J' , 'I' , 'X' , 'P') 
group by 1
having order_count>=1
),
promo_order as (
select customer_id
from cdm.order_line_promotion_usage
where promotion_id = 12105
and customer_id in  (select distinct customer_id from control_customer )
and order_status not in ('J' , 'I' , 'X' , 'P') 
),
unique_users as (
select 
customer_id, 
count(distinct session_id) as session_cnt,
count(distinct case when HOME_PAGE_IMPRESSION_HITS >0 then session_id end ) impression_sessions
from EDLDB.ECOM_SANDBOX.HEARTBEAT_SESSION_AGGREGATE
where session_date between $start_date and $end_date
and customer_id in (select distinct customer_id from control_customer)
group by 1
),
customer_data as (
select distinct a.customer_id, a.regular_flag, a.consistent_flag, a.buckets , a.updated_flag ,
case when b.customer_id is null then False else True end as SESSION_FLAG,
case when b.customer_id is null or b.impression_sessions = 0 then False else True end as HP_FLAG,
case when d.customer_id is null then False else True end as ORDER_FLAG,
case when e.customer_id is null then False else True end as SW_ORDER_FLAG

from control_customer a
left join unique_users b on a.customer_id = b.customer_id
left join order_check d on a.customer_id = d.customer_id
left join promo_order e on a.customer_id = e.customer_id
)

select
a.regular_flag, 
a.consistent_flag, 
a.buckets ,
a.SESSION_FLAG,
a.HP_FLAG,
a.ORDER_FLAG,
a.SW_ORDER_FLAG,
count(distinct customer_id) as customer_count

from customer_data a 
group by 1,2,3,4,5,6,7
;







"""